In [24]:
import os
import pandas as pd
import numpy as np

In [25]:
input_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\original\anning_jijianrong'
input_files = os.listdir(input_folder)
output_folder = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\derived'

In [26]:
df = pd.read_excel(os.path.join(input_folder, input_files[0]))
df.head(3)

,PID,冈上,Unnamed: 2,Unnamed: 3,冈下,Unnamed: 5,Unnamed: 6,肩胛下,Unnamed: 8,Unnamed: 9
0,状态,无撕裂,部分撕裂,完全撕裂,无撕裂,部分撕裂,完全撕裂,无撕裂,部分撕裂,完全撕裂
1,1002088,1,5,1,5,1,1,5,1,1
2,1002469,1,2,4,5,1,1,5,1,1


In [27]:
def tear_result(results):
    max_result = np.max(results)
    max_loc = np.where(results==max_result)
    return np.mean(max_loc)


for input_file in input_files:
    reader_file = os.path.join(input_folder, input_file)
    df = pd.read_excel(reader_file)
    supraspinatus, infraspinatus, subscapularis = ['NA'],['NA'],['NA']
    for index, row in df.iterrows():
        if index:
            supraspinatus.append(tear_result(row[1:4]))
            infraspinatus.append(tear_result(row[4:7]))
            subscapularis.append(tear_result(row[7:10]))
    df['supraspinatus'] = supraspinatus
    df['infraspinatus'] = infraspinatus
    df['subscapularis'] = subscapularis

    df.to_excel(os.path.join(output_folder, input_file), index=False)

   
    
        
        

## concat results

In [28]:
input_root = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\derived'
output_root = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\shoulder_mr\reader_results\final'

In [29]:
df_temp = pd.DataFrame()

for reader in os.listdir(input_root):
    reader_df = pd.DataFrame()
    cur_reader = os.path.join(input_root, reader)
    for result_file in os.listdir(cur_reader):
        cur_result_file = os.path.join(cur_reader, result_file)
        temp_df = pd.read_excel(cur_result_file)
        reader_df = pd.concat([reader_df, temp_df.iloc[1:, [0,10,11,12]]])
    reader_df.to_excel(os.path.join(output_root, f'{reader}.xlsx'),index=False)    
                        
        




In [21]:
temp_df.iloc[1:,[0,10,11,12]]

,PID,Supraspinatus,Infraspinatus,Subscapularis
1,1002088,1.0,0.0,0.0
2,1002469,1.0,0.0,0.0
3,1003152,2.0,0.5,0.5
4,1004021,2.0,0.0,0.0
5,1004365,1.5,0.0,0.0
...,...,...,...,...
468,1674501,0.0,0.0,0.0
469,1675122,0.0,0.0,0.0
470,1675133,2.0,0.0,0.0
471,1675516,1.0,0.0,0.0


In [6]:
import SimpleITK as sitk
import numpy as np

In [7]:
test_file = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\huashan_LLM\xie.nii.gz'
test_dicom = r'C:\Users\z004b0je\Downloads\腰椎DIXON-axi\腰椎DIXON-axi\术后Dixon\Q-Dixon (axi)_FF_205938_29\00000043.dcm'

In [8]:
mask_image = sitk.ReadImage(test_file)
image = sitk.ReadImage(test_dicom)

In [46]:
import math
import SimpleITK as sitk
import numpy as np

mask_file = r'C:\Users\z004b0je\OneDrive - Siemens Healthineers\Projects\20_Scientific_Proof_Points\huashan_LLM\xie.nii.gz'
image_dicom = r'C:\Users\z004b0je\Downloads\腰椎DIXON-axi\腰椎DIXON-axi\术后Dixon\Q-Dixon (axi)_FF_205938_29\00000043.dcm'

mask_image = sitk.ReadImage(mask_file)
image = sitk.ReadImage(image_dicom)

spacing = image.GetSpacing()
size = image.GetSize()

def circle_coordinates(center_x, center_y, radius):

    theta = 0.0
    increment = 0.01  # 步长，决定点的密度
    coordinates = []
    
    while theta < 2 * math.pi:
        x = round(center_x + radius * math.cos(theta))
        y = round(center_y + radius * math.sin(theta))
        coordinates.append((x, y))
        theta += increment
        
    return coordinates

x = 90 # CoR
y = 94


img = sitk.GetArrayFromImage(image).reshape(size[0],size[1])
muscle_img = [0]
for i in range(3):
    mask_img = sitk.GetArrayFromImage(mask_image)[20].reshape(size[0],size[1])
    mask_img[mask_img != (i+1)] = 0
    muscle_img.append(img * mask_img)
    
muscle_img1 = muscle_img[1] 
fi = dict()
for r_mm in range(18,64):
    intensities = []
    r = round(r_mm/spacing[0])
    coords = set(circle_coordinates(y,x,r))

    for coord in coords:
        intensities.append(muscle_img1[coord[1],coord[0]])
    fi.setdefault(r_mm,intensities)
for key in fi:
    print(f'Radius {key}, FI% {round(np.mean(np.array(fi[key])),1)}')  

Radius 18, FI% 6.1
Radius 19, FI% 6.1
Radius 20, FI% 6.5
Radius 21, FI% 6.5
Radius 22, FI% 8.0
Radius 23, FI% 8.5
Radius 24, FI% 8.5
Radius 25, FI% 11.6
Radius 26, FI% 9.3
Radius 27, FI% 9.3
Radius 28, FI% 8.0
Radius 29, FI% 13.9
Radius 30, FI% 13.9
Radius 31, FI% 6.1
Radius 32, FI% 6.1
Radius 33, FI% 4.4
Radius 34, FI% 6.3
Radius 35, FI% 6.3
Radius 36, FI% 4.9
Radius 37, FI% 6.1
Radius 38, FI% 6.1
Radius 39, FI% 4.2
Radius 40, FI% 3.5
Radius 41, FI% 3.5
Radius 42, FI% 6.5
Radius 43, FI% 7.8
Radius 44, FI% 7.8
Radius 45, FI% 9.4
Radius 46, FI% 9.4
Radius 47, FI% 8.4
Radius 48, FI% 2.8
Radius 49, FI% 2.8
Radius 50, FI% 4.5
Radius 51, FI% 4.0
Radius 52, FI% 4.0
Radius 53, FI% 5.5
Radius 54, FI% 4.2
Radius 55, FI% 4.2
Radius 56, FI% 3.7
Radius 57, FI% 3.7
Radius 58, FI% 3.6
Radius 59, FI% 4.4
Radius 60, FI% 4.4
Radius 61, FI% 4.5
Radius 62, FI% 2.9
Radius 63, FI% 2.9


In [36]:
import seaborn as sns


18 0.0
19 0.0
20 0.0
21 0.0
22 0.0
23 0.0
24 0.0
25 0.0
26 0.0
27 0.0
28 0.0
29 28.6
30 28.6
31 35.87755102040816
32 35.87755102040816
33 47.0377358490566
34 33.42603550295858
35 33.42603550295858
36 27.344444444444445
37 22.410404624277458
38 22.410404624277458
39 17.96315789473684
40 17.67
41 17.67
42 18.564356435643564
43 16.876847290640395
44 16.876847290640395
45 16.317307692307693
46 16.317307692307693
47 20.969026548672566
48 19.378854625550662
49 19.378854625550662
50 12.665178571428571
51 10.265060240963855
52 10.265060240963855
53 20.552419354838708
54 24.773234200743495
55 24.773234200743495
56 22.78125
57 22.78125
58 17.935606060606062
59 22.397849462365592
60 22.397849462365592
61 19.029520295202953
62 6.248322147651007
63 6.248322147651007


In [17]:
coord

(106.0, 90.0)